Chinese stopwords: https://github.com/stopwords-iso/stopwords-zh

In [1]:
import pandas as pd 
import spacy
import numpy as np


In [2]:
weibo_df = pd.read_csv("C:/Users/kgk/OneDrive - Aalborg Universitet/CALDISS_projects/digital-literacy_E20/text-networks/data/2019-12.csv")

In [3]:
nlp = spacy.load("zh_core_web_sm")

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\kgk\AppData\Local\Temp\jieba.cache
Loading model cost 3.095 seconds.
Prefix dict has been built successfully.


In [21]:
testdoc = nlp(weibo_df.loc[2, 'content'])[0:10]

In [23]:
testdoc

明天新的《中华人民共和国药品管理法》

In [22]:
testdoc.sentiment

0.0

In [25]:
def tokenizer_custom(text):
    doc = nlp(text)
    
    pos_tags = ['PROPN', 'ADJ', 'NOUN']
    
    tokens = []
      
    for token in doc:
        if word.pos in pos_tags:
            tokens.append(word.text)
    
    return(tokens)

In [59]:
def mean_sentiment(text):
    doc = nlp(text)
    
    mean_sentiment = np.mean([sentence.sentiment for sentence in doc.sentences])
    
    return(mean_sentiment)

In [61]:
weibo_subset = weibo_df.loc[weibo_df['like_num'] > 0, :]

In [63]:
weibo_subset.shape

(11209, 9)

In [ ]:
#weibo_subset.loc[:, 'tokens'] = weibo_subset.loc[:,'content'].apply(tokenizer_custom)

weibo_subset.loc[:, 'sentiment'] = weibo_subset.loc[:, 'content'].apply(mean_sentiment)

In [ ]:
weibo_subset['created_at'] = pd.to_datetime(weibo_subset['created_at'])

In [ ]:
weibo_subset.head()

### Visualizing

In [ ]:
import seaborn as sns

sns.scatterplot(data = weibo_df, x = 'created_at', y = 'like_num', hue = 'sentiment')

### Summarizing by week

C:\Users\kgk\.conda\envs\tm\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
weibo_subset['post_week'] = weibo_subset['created_at'].isocalendar().week

weibo_grouped = weibo_subset.groupby('post_week')

weibo_grouped['sentiment'].mean()